# Comparison between Discrete and Fast Fourier Transform.
#### Following JVP's online tutorial

In [2]:
import numpy as np

In [3]:
def DFT_slow(x):
    """
    Compute the discrete Fourier Transform of the 1D array x.
    This is a slow-running code on purpose.
    """
    
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    n = np.arange(N)
    k = n.reshape((N, 1))
    M = np.exp(-2j * np.pi * k * n / N) #Standard Fourier Transform equation
    
    return np.dot(M, x)

In [4]:
#I can check that function works against numpy's built in fft function
x = np.random.random(1024)
np.allclose(DFT_slow(x), np.fft.fft(x))

True

We can speed up the DFT process by exploiting symmetry in what's known as FFT.

Essentially, $X_{N+k}=X_{k}$. 

We can further the argument to complex equivalencies: $X_{iN+k}=X_{k}$

This allows us to split Fourier Transform integrals into two. One for even integers and one for odd integers. We can see from symmetry that we would only need one of these, and we can therefore cut our computation in half.

In [15]:
def FFT(x):
    """
    A recursive implementation of the 1D Cooley-Tukey FFT by JVP.
    """
    x = np.asarray(x, dtype=float)
    N = x.shape[0]
    
    if N % 2 > 0:
        raise ValueError("size of x must be a power of 2")
    elif N <= 30:  # This cutoff has ben optimpized for the random array, but can be switched for our data
        return DFT_slow(x)
    else:
        X_even = FFT(x[::2]) #This is a very matlab logic argument
        X_odd = FFT(x[1::2])
        factor = np.exp(-2j * np.pi * np.arange(N) / N) #The fourier transform equation
        return np.concatenate([X_even + factor[:N // 2] * X_odd,
                               X_even + factor[N // 2:] * X_odd]) #The equation solves N/2 for the odd numbers

In [16]:
#Checking that this functions works
np.allclose(FFT(x), np.fft.fft(x))

True

In [19]:
#Now I can check if the timing changes!

%timeit DFT_slow(x)

10 loops, best of 3: 84.4 ms per loop


In [20]:
%timeit FFT(x)

100 loops, best of 3: 4.56 ms per loop


In [21]:
%timeit np.fft.fft(x)

The slowest run took 23.66 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 43.2 µs per loop


Numpy's fft is clearly the fastest.

In [ ]:
def fft(a, n=None, axis=-1, norm=None):
    """
    Compute the one-dimensional discrete Fourier Transform.
    This function computes the one-dimensional *n*-point discrete Fourier
    Transform (DFT) with the efficient Fast Fourier Transform (FFT)
    algorithm [CT].
    Parameters

    Notes
    -----
    FFT (Fast Fourier Transform) refers to a way the discrete Fourier
    Transform (DFT) can be calculated efficiently, by using symmetries in the
    calculated terms.  The symmetry is highest when `n` is a power of 2, and
    the transform is therefore most efficient for these sizes.
    The DFT is defined, with the conventions used in this implementation, in
    the documentation for the `numpy.fft` module.
    References
    ----------
    .. [CT] Cooley, James W., and John W. Tukey, 1965, "An algorithm for the
            machine calculation of complex Fourier series," *Math. Comput.*
            19: 297-301.
            
    Examples
    --------
    >>> np.fft.fft(np.exp(2j * np.pi * np.arange(8) / 8))
    array([ -3.44505240e-16 +1.14383329e-17j,
             8.00000000e+00 -5.71092652e-15j,
             2.33482938e-16 +1.22460635e-16j,
             1.64863782e-15 +1.77635684e-15j,
             9.95839695e-17 +2.33482938e-16j,
             0.00000000e+00 +1.66837030e-15j,
             1.14383329e-17 +1.22460635e-16j,
             -1.64863782e-15 +1.77635684e-15j])
    In this example, real input has an FFT which is Hermitian, i.e., symmetric
    in the real part and anti-symmetric in the imaginary part, as described in
    the `numpy.fft` documentation:
    >>> import matplotlib.pyplot as plt
    >>> t = np.arange(256)
    >>> sp = np.fft.fft(np.sin(t))
    >>> freq = np.fft.fftfreq(t.shape[-1])
    >>> plt.plot(freq, sp.real, freq, sp.imag)
    [<matplotlib.lines.Line2D object at 0x...>, <matplotlib.lines.Line2D object at 0x...>]
    >>> plt.show()
    """

    a = np.asarray(a).astype(complex, copy=False)
    if n is None:
        n = a.shape[axis]
    output = _raw_fft(a, n, axis, fftpack.cffti, fftpack.cfftf, _fft_cache) #must look closer at these packs
    if _unitary(norm):
        output *= 1 / sqrt(n)
    return output